## Langchain Agent Executor From Scratch in Langgraph

In [4]:
!pip install --quiet -U langgraph langchain vertexai langchain_google_vertexai langchainhub tavily-python


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import arxiv
from IPython.display import display, Markdown
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_vertexai import ChatVertexAI
import vertexai
from langchain_core.messages import HumanMessage
from vertexai.generative_models._generative_models import (
    HarmCategory,
    HarmBlockThreshold,
)
from google.cloud.aiplatform_v1beta1.types import (
    content as gapic_content_types,
)
from vertexai.generative_models import GenerativeModel,FunctionDeclaration,Tool,GenerationConfig,Part,Content
from vertexai.preview.generative_models import ToolConfig
from langchain.tools.render import format_tool_to_openai_function


vertexai.init(project="gemini-api-428204", location="us-central1")

In [3]:
search_arxiv = FunctionDeclaration(
    name="search_arxiv",
    description="Search for articles and publications in arXiv",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "Query to search for in arXiv"}
        },
    },
)


In [4]:
search_tool = Tool(
    function_declarations=[
        search_arxiv,
    ],
)


In [5]:
        get_current_weather_func = FunctionDeclaration(
            name="get_current_weather",
            description="Get the current weather in a given location",
            parameters={
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": [
                            "celsius",
                            "fahrenheit",
                        ]
                    }
                },
                "required": [
                    "location"
                ]
            },
        )
        weather_tool = Tool(
            function_declarations=[get_current_weather_func],
        )


In [90]:
prompt = "What is the weather in san francisco today?"
model = GenerativeModel("gemini-pro",tools=[weather_tool])
response_function_call_content = response.candidates[0].content
response = model.generate_content("What is the weather in san francisco today")
print(response.candidates[0].content.parts[0].function_call)
print(response_function_call_content)

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "san francisco, ca"
    }
  }
}

role: "model"
parts {
  function_call {
    name: "get_current_weather"
    args {
      fields {
        key: "location"
        value {
          string_value: "San Francisco, CA"
        }
      }
    }
  }
}



In [91]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key] = value
params


{'location': 'san francisco, ca'}

In [92]:
def get_weather(location: str):
    return f"The weather of {location} is 82"

In [93]:
weather_response = get_weather(params["location"])

In [95]:
response = model.generate_content(
    [
        Content(
            role="user",
            parts=[
                Part.from_text(prompt),
            ],
        ),
        response_function_call_content,  # Function call response
        Content(
            parts=[
                Part.from_function_response(
                    name="get_current_weather",
                    response={
                        "content": weather_response,  # Return the API response to the Gemini model
                    },
                )
            ],
        ),
    ],
    tools=[weather_tool],
)

display(Markdown(response.text))


The current weather in San Francisco, CA is 82. 


In [108]:
tool_config = ToolConfig(
    function_calling_config=ToolConfig.FunctionCallingConfig(
        mode=ToolConfig.FunctionCallingConfig.Mode.AUTO,  # The default model behavior. The model decides whether to predict a function call or a natural language response.
    )
)


In [ ]:
prompt = "Explain the Schrodinger equation in a few sentences and give me papers from arXiv to learn more"
response = model.generate_content(prompt, tools=[search_tool], tool_config=tool_config)

display(Markdown(response.candidates[0].content.parts[0].text))


In [50]:
tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

safety_settings = {
	HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
	HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

# Choose the LLM that will drive the agent
llm = ChatVertexAI(model_name="gemini-pro",additional_kwargs={"safety_settings": safety_settings,"generation_config":gapic_content_types.GenerationConfig(
                temperature=0.1,
                top_p=0.95,
                top_k=20,
                candidate_count=1,
                max_output_tokens=100,
                stop_sequences=["STOP!"],
            )})

functions = [format_tool_to_openai_function(t) for t in tools]
model = llm.bind_tools(functions)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)


In [57]:
res = model.invoke( [HumanMessage(content="What is Langchain")])

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

In [55]:
print(res)

content='' additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is LLM"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 51, 'candidates_token_count': 13, 'total_token_count': 64}} id='run-6331b89e-1e3a-4351-91d1-17f3b066e6db-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is LLM'}, 'id': '0f29c9d4-f96a-488a-8288-0c3a55eba241'}] usage_metadata={'input_tokens': 51, 'output_tokens': 13, 'total_tokens': 64}


In [53]:
toolT = TavilySearchResults(max_results=1)
toolT.invoke(res.additional_kwargs["function_call"]['arguments'])

KeyError: 'function_call'

In [34]:
resp = agent_runnable.invoke({"input": "What is BIM","intermediate_steps":[]})

In [35]:
print(resp)

tool='tavily_search_results_json' tool_input={'query': 'What is BIM'} log="\nInvoking: `tavily_search_results_json` with `{'query': 'What is BIM'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is BIM"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 55, 'candidates_token_count': 12, 'total_token_count': 67}}, id='run-0e1f7bd3-cd37-4a14-a10a-0da761d7233a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is BIM'}, '

In [19]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


In [36]:
from langchain_core.agents import AgentFinish

from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)


def run_agent(data):
    print(data)
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


def execute_tools(data):
    agent_action = data["agent_outcome"]
    print("--------------------agent_action",agent_action)
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"


In [58]:
tool_executor.invoke(resp)

[{'url': 'https://www.pbctoday.co.uk/news/digital-construction-news/bim-news/unlock-power-bim-field-with-reality-capture/140022/',
  'content': 'How Digital Building Logbooks can boost retrofit and sustainability The potential of BIM for renovation projects The golden age of AI-based construction management and site monitoring From concept to construction: building with connected digital workflows United in the delivery of responsible AI Tap into technology to bridge the construction skills gap A look inside the Digital Construction Week 2024 speaker programme How digital materials procurement promotes sustainable practices A guide to thermal insulation standards in the UK Construction materials procurement transformed with data analytics LEAVE A REPLY Cancel reply Save my name, email, and website in this browser for the next time I comment.  Unlock the power of BIM in the field with reality capture OpenSpace, a leading AI construction tech company, enables field teams to navigate BIM 

In [37]:
from langgraph.graph import END, StateGraph, START

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()


In [40]:
app.invoke(
    {"input": "What is BIM","chat_history": []},
    config={"configurable": {"thread_id": 42}}
)


{'input': 'What is BIM', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': []}
--------------------agent_action tool='tavily_search_results_json' tool_input={'query': 'What is BIM?'} log="\nInvoking: `tavily_search_results_json` with `{'query': 'What is BIM?'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is BIM?"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 55, 'candidates_token_count': 13, 'total_token_count': 68}}, id='r

{'input': 'What is BIM',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': '## BIM: Building Information Modeling\n\nBuilding Information Modeling (BIM) is a process for creating and managing information about a building or other physical object across its entire lifecycle. This information can be used to generate 3D models, 2D drawings, and other documentation that can be used for design, construction, and operation of the building.\n\nHere are some key points about BIM:\n\n* **Digital Representation:** BIM creates a digital representation of a building, including all of its physical and functional characteristics. This model can be used to simulate how the building will perform under different conditions, and to generate accurate construction documents.\n* **Collaboration:** BIM facilitates collaboration between different disciplines involved in the design and construction process. This is because the model provides a single source of truth for all project i